In [ ]:
!pip install d2l==1.0.3
! pip install biopython
!pip install wandb -qU
!pip install focal_loss_torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.0/95.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [ ]:
import os
! git clone https://git.wur.nl/bioinformatics/grs34806-deep-learning-project-data.git
os.chdir("grs34806-deep-learning-project-data")

Cloning into 'grs34806-deep-learning-project-data'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21 (from 1)
Receiving objects: 100% (21/21), 8.74 MiB | 6.25 MiB/s, done.


In [ ]:
import math
import numpy as np
import wandb

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split, RandomSampler, WeightedRandomSampler

from d2l import torch as d2l

import matplotlib.pyplot as plt

from sklearn.metrics import f1_score, confusion_matrix

from focal_loss.focal_loss import FocalLoss

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Functions in this codeblock created with the help of template provided in the Deep Learning (GRS34806), Bioinformatics project, 2025
def read(seqfile, posfile):
  # seqfile: file with sequences
  # posfile: file with positive cases (annotated with function)
  # … add your code here
  datalist = []
  annot_id_list = []
  labellist = []
  with open(seqfile) as file:
      for line in file:
        label, seq = (line.split())
        annot_id_list.append(label)
        datalist.append(seq)

  pos_annot_indices = []

  with open(posfile) as file:
      for line in file:
        annotation = (line.strip())
        try:
          pos_annot_indices.append(annot_id_list.index(annotation))
        except:
          continue

  labellist = [1 if i in pos_annot_indices else 0 for i, x in enumerate(datalist)]

  # datalist contains sequences, labellist contains labels
  return datalist, labellist


def generate_train_test(datalist, labellist, test_prop=0.3):
  # Split up dataset in training set and test set
  data_len = len(datalist)
  test_set_size = math.floor(data_len * test_prop)
  test_set_indices = np.random.choice(np.arange(data_len), size=test_set_size, replace=False)
  test_set_mask = np.zeros(data_len)
  test_set_mask[test_set_indices] = 1
  train_set_mask = 1 - test_set_mask

  traindatalist = np.array(datalist)[np.where(train_set_mask == 1)]
  trainlabellist = np.array(labellist)[np.where(train_set_mask == 1)]

  testdatalist = np.array(datalist)[np.where(test_set_mask == 1)]
  testlabellist = np.array(labellist)[np.where(test_set_mask == 1)]

  return (traindatalist.tolist(), trainlabellist.tolist()), (testdatalist.tolist(), testlabellist.tolist())



def load_data(batch_size, num_steps, dataset, sampler=None):

  mapaa2num = {aa: i for (i, aa)
                      in enumerate(list("ACDEFGHIKLMNPQRSTVWY"))}

  seq, lab = dataset
  seq = tokenise(seq, mapaa2num)
  seq_array = build_seq_array(seq, num_steps)

  seq_array = F.one_hot(seq_array, num_classes=21)

  seq_array = seq_array.type(torch.float32)
  seq_array = seq_array.permute(0, 2, 1)

  # Create TensorDataset
  dataset = TensorDataset(seq_array, torch.tensor(lab).unsqueeze(1).type(torch.float32))

  if sampler:
    # Create DataLoader
    data_iter = DataLoader(dataset, batch_size=batch_size, sampler=sampler)
  else:
    data_iter = DataLoader(dataset, batch_size=batch_size, shuffle=True)

  return data_iter


def tokenise(sequences,map2num,non_aa_num=20):
  tokenised_seq = []
  for count, i in enumerate(sequences):
    tokenised_seq.append([map2num.get(j,non_aa_num) for j in list(i)])
  return tokenised_seq

def build_seq_array(tokenised_seq, num_steps, pad_token=20):
  array = torch.tensor([
      truncate_pad(seq, num_steps, pad_token) for seq in tokenised_seq])
  return array


def truncate_pad(seq_tokens, num_steps, padding_token):
  if len(seq_tokens) > num_steps:
    return seq_tokens[:num_steps]  # Truncate
  return seq_tokens + [padding_token] * (num_steps - len(seq_tokens))  # Pad


In [ ]:
class SimpleCnn(nn.Module):
    """Full transformer architecture, but without optimizations such as batching or dropout"""
    def __init__(self, dropout: float=0.2, number_of_layers: int=2, final_output_number: int=3):
        super().__init__()
        self.loss_fn = nn.BCELoss()

        layers = []
        input_number = 21
        output_number = final_output_number * number_of_layers
        for i in range(number_of_layers):
          layers.append(self.make_layer(dropout, input_number, output_number))
          input_number = output_number
          output_number = output_number - final_output_number

        self.net = nn.Sequential(
          *layers,

          nn.Flatten(),

          nn.LazyLinear(1),
          nn.Sigmoid()
        )

    def make_layer(self, dropout, input_number, output_number):
      layer = nn.Sequential(
          nn.Conv1d(input_number, output_number, kernel_size=3, padding=2),
          nn.ReLU(),
          nn.Dropout(dropout)
      )
      return layer


    def forward(self, x: torch.Tensor, targets: torch.Tensor=None) -> tuple[torch.Tensor, torch.Tensor | None, torch.Tensor | None]:
        """Predict class logits and calculate loss and accuracy if targets are specified"""
        prediction = self.net(x)
        loss = self.loss_fn(prediction, targets)
        predicted_labels = (prediction > 0.5).float()
        accuracy = (predicted_labels == targets).sum() / targets.size(0)

        return prediction, loss, accuracy

In [ ]:
class SimpleCnnWithPool(nn.Module):
    """Full transformer architecture, but without optimizations such as batching or dropout"""
    def __init__(self, dropout: float=0.2, number_of_layers: int=2, final_output_number: int=3):
        super().__init__()
        self.loss_fn = nn.BCELoss()

        layers = []
        input_number = 21
        output_number = final_output_number * number_of_layers
        for i in range(number_of_layers):
          layers.append(self.make_layer(dropout, input_number, output_number))
          input_number = output_number
          output_number = output_number - final_output_number


        self.net = nn.Sequential(
          *layers,
          nn.Flatten(),

          nn.LazyLinear(1),
          nn.Sigmoid()
        )

    def make_layer(self, dropout, input_number, output_number):
      layer = nn.Sequential(
          nn.Conv1d(input_number, output_number, kernel_size=3, padding=2),
          nn.ReLU(),
          nn.MaxPool1d(kernel_size=3, stride=2, padding=1),
          nn.Dropout(dropout)
      )
      return layer

    def forward(self, x: torch.Tensor, targets: torch.Tensor=None) -> tuple[torch.Tensor, torch.Tensor | None, torch.Tensor | None]:
        """Predict class logits and calculate loss and accuracy if targets are specified"""
        prediction = self.net(x)
        loss = self.loss_fn(prediction, targets)
        predicted_labels = (prediction > 0.5).float()
        accuracy = (predicted_labels == targets).sum() / targets.size(0)

        return prediction, loss, accuracy



In [ ]:
class SimpleCnnWithPool_FocalLoss(nn.Module):
    """Full transformer architecture, but without optimizations such as batching or dropout"""
    def __init__(self, dropout: float=0.2, number_of_layers: int=2, final_output_number: int=3, focal_alpha: torch.Tensor=None, gamma: int=0):
        super().__init__()
        self.loss_fn = FocalLoss(weights=focal_alpha, gamma=gamma)

        layers = []
        input_number = 21
        output_number = final_output_number * number_of_layers
        for i in range(number_of_layers):
          layers.append(self.make_layer(dropout, input_number, output_number))
          input_number = output_number
          output_number = output_number - final_output_number


        self.net = nn.Sequential(
          *layers,
          nn.Flatten(),

          nn.LazyLinear(1),
          nn.Sigmoid()
        )

    def make_layer(self, dropout, input_number, output_number):
      layer = nn.Sequential(
          nn.Conv1d(input_number, output_number, kernel_size=3, padding=2),
          nn.ReLU(),
          nn.MaxPool1d(kernel_size=3, stride=2, padding=1),
          nn.Dropout(dropout)
      )
      return layer

    def forward(self, x: torch.Tensor, targets: torch.Tensor=None) -> tuple[torch.Tensor, torch.Tensor | None, torch.Tensor | None]:
        """Predict class logits and calculate loss and accuracy if targets are specified"""
        prediction = self.net(x)
        targets = targets.type(torch.LongTensor)
        loss = self.loss_fn(prediction.squeeze().to(DEVICE), targets.squeeze().to(DEVICE))
        predicted_labels = (prediction > 0.5).float()
        accuracy = (predicted_labels.to('cpu') == targets).sum() / targets.size(0)
        return prediction, loss, accuracy



In [ ]:
# This function is adopted from Weights and Biases: https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb
def get_optimizer(network, optimizer, learning_rate, momentum):
  if optimizer == "sgd":
    optimizer = torch.optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=momentum)
  elif optimizer == "adam":
    optimizer = torch.optim.Adam(network.parameters(),
                               lr=learning_rate)
  return optimizer


# For this function we used gemini for debugging
def train_net(config=None):
    """Train the network with specified parameters."""


    # Initialize wandb
    with wandb.init(config=config) as run:
      config = wandb.config

      seq_file = file_names[config.file_name]['seq']
      pos_file = file_names[config.file_name]['pos']


      lr = round(config.learning_rate, 3)
      # Create a custom name from hyperparameters
      run.name = f"lr_{lr}_ln_{config.layer_number}"
      run.save()

      # Set device
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

      # Load data
      dataset = read(seq_file, pos_file)
      train_data, test_data = generate_train_test(*dataset)

      max_len = 0
      for seq in train_data[0]:
        if len(seq) > max_len:
          max_len = len(seq)

      if config.use_weighted_sampler:
        print(sum(train_data[1]))
        class_sample_count = np.array([len(np.where(np.array(train_data[1]) == t)[0]) for t in np.unique(train_data[1])])
        weight = 1. / class_sample_count
        samples_weight = np.array([weight[t] for t in train_data[1]])
        samples_weight = torch.from_numpy(samples_weight)
        print(len(samples_weight))
        sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight), replacement=True)

        train_loader = load_data(config.batch_size, max_len, train_data, sampler=sampler)
      else:
        train_loader = load_data(config.batch_size, max_len, train_data)
      test_loader  = load_data(config.batch_size, max_len, test_data)

      print("Checking class label counts in train_loader:")
      label_counts = {0: 0, 1: 0}  # Initialize counts for each class
      for _, labels in train_loader:
          for label in labels.flatten():
              label_counts[label.item()] += 1
      print(f"Class 0 count: {label_counts[0]}")
      print(f"Class 1 count: {label_counts[1]}")
      print("-" * 20)  # Separator

      model = architectures[config.net](config.dropout, number_of_layers=config.layer_number).to(device)

      optimizer = get_optimizer(model, config.optimizer, lr, config.momentum)

      for epoch in range(config.epochs):
          # Training loop for one epoch
          model.train()  # Set the model to training mode
          all_x = []
          all_y = []
          for x, y in train_loader:
              all_x.append(x.to(device))
              all_y.append(y.to(device))

          all_x = torch.cat(all_x, dim=0)
          all_y = torch.cat(all_y, dim=0)

          optimizer.zero_grad()  # Reset gradients
          pred, loss, accuracy = model(all_x, all_y)  # Forward pass for all data
          loss.backward()  # Backpropagate gradients
          optimizer.step()  # Update model parameters

          predicted_labels = (pred > 0.5).float()
          train_f1 = f1_score(all_y.cpu().numpy(), predicted_labels.cpu().numpy())

          # Evaluation after each epoch
          model.eval()
          with torch.no_grad():  # Disable gradient calculation
              all_test_x = []
              all_test_y = []
              for x, y in test_loader:
                  all_test_x.append(x.to(device))
                  all_test_y.append(y.to(device))

              all_test_x = torch.cat(all_test_x, dim=0)
              all_test_y = torch.cat(all_test_y, dim=0)

              test_pred, test_loss, test_accuracy = model(all_test_x, all_test_y)

          predicted_labels = (test_pred > 0.5).float()
          test_f1 = f1_score(all_test_y.cpu().numpy(), predicted_labels.cpu().numpy())

          wandb.log({
              "epoch" : epoch,
              "train_loss": loss.item(),
              "test_loss": test_loss.item(),
              "train_accuracy": accuracy.item(),
              "test_accuracy": test_accuracy.item(),
              "train_f1": train_f1,
              "test_f1": test_f1,
          })


In [ ]:
architectures = {
    'SimpleCnn': SimpleCnn,
    'SimpleCnnWithPool': SimpleCnnWithPool
}

file_names = {
  'len100_200_n1000': {
      'seq': 'len100_200_n1000.seq',
      'pos': 'len100_200_n1000.pos'
  },
  'len200_500_n1000': {
      'seq': 'len200_500_n1000.seq',
      'pos': 'len200_500_n1000.pos'
  },
  'len200_500_n1000_nr2': {
      'seq': 'len200_500_n5000nr2.seq',
      'pos': 'len200_500_n5000nr2.pos'
  },
  'len200_500_n1000_nr3': {
      'seq': 'len200_500_n5000nr3.seq',
      'pos': 'len200_500_n5000nr3.pos'
      },

  "human_training_GO_3A0005576": {
      'seq': "expr5Tseq_filtGO_100-1000.lis",
      'pos': "GO_3A0005576.annotprot"
      },
  "human_training_GO_3A0005739": {
      'seq': "expr5Tseq_filtGO_100-1000.lis",
      'pos': "GO_3A0005739.annotprot"
      },
  "human_training_GO_3A0007165": {
      'seq': "expr5Tseq_filtGO_100-1000.lis",
      'pos': "GO_3A0007165.annotprot"
      },
  "human_training_GO_3A0043066": {
      'seq': "expr5Tseq_filtGO_100-1000.lis",
      'pos': "GO_3A0043066.annotprot"
      },
  "human_training_GO_3A0055085": {
      'seq': "expr5Tseq_filtGO_100-1000.lis",
      'pos': "GO_3A0055085.annotprot"
      },

}


In [ ]:
sweep_config = {
    'name': 'sweep_len100_200_n1000',
    'method': 'random',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
        }
    }

parameters_dict = {
    'net': {
        'values': ['SimpleCnn', 'SimpleCnnWithPool']
        },
    'layer_number': {
        'values': [2, 3, 4, 5]
    },
    'use_weighted_sampler': {
        'value': False
    },
    "file_name": {
        "value": "len100_200_n1000"
        },
    "optimizer": {
        "values": ["sgd", "adam"],
        },
    "learning_rate": {
        "distribution": "uniform",
        "min": 0.001,
        "max": 0.1
        },
    "weight_decay": {
        "distribution": "log_uniform_values",
        "min": 1e-5,
        "max": 1e-1
        },
    "dropout": {
        "distribution": "uniform",
        "min": 0.2,
        "max": 0.5
        },
    "batch_size": {
        "values": [128, 256, 512]
        },
    "epochs": {
      "value": 100
      },
    "momentum": {
        "values": [0.8, 0.9, 0.99],
        }
    }

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="deep-learning-MBF4")
wandb.agent(sweep_id, train_net, count=100)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: atbgmvvz
Sweep URL: https://wandb.ai/n-j-hindriks-wur/deep-learning-MBF4/sweeps/atbgmvvz


wandb: Agent Starting Run: exn1m9g0 with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.40617645729256296
wandb: 	epochs: 100
wandb: 	file_name: len100_200_n1000
wandb: 	layer_number: 3
wandb: 	learning_rate: 0.02027974903283214
wandb: 	momentum: 0.8
wandb: 	net: SimpleCnn
wandb: 	optimizer: sgd
wandb: 	use_weighted_sampler: False
wandb: 	weight_decay: 0.00014887823767443446
wandb: Currently logged in as: n-j-hindriks (n-j-hindriks-wur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Checking class label counts in train_loader:
Class 0 count: 336
Class 1 count: 364
--------------------


...

Checking class label counts in train_loader:
Class 0 count: 344
Class 1 count: 356
--------------------


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇██
test_accuracy,▇▁▇█████████████████████████████████████
test_f1,▁▁▂▅▇███████████████████████████████████
test_loss,█▆▆▅▄▁▁▁▁▁▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇
train_accuracy,▆▄▄▄▄▇▁▄▃▆▅▅▆▃▆▄▄▇▅▅▆▆▅▅▆▄▅▆▇▇▆▃▅▃█▅▄▄▄▇
train_f1,▁▁▄▃▄▆▆▇▇▇▇█▇█▇▇█▇▇██▇██▇▇▇███▇█████▇███
train_loss,▂▂▃▅▅▅█▅▆▂▅▄▃▂▃▄█▄▂▃▄▂▄▃▆▂▂▁▆▃▃▄▃▅▄▅▂▄▅▃
epoch,99
test_accuracy,0.50333
test_f1,0.66962
test_loss,0.69339


wandb: Agent Starting Run: 6y2vfu7b with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.479812234972956
wandb: 	epochs: 100
wandb: 	file_name: len100_200_n1000
wandb: 	layer_number: 5
wandb: 	learning_rate: 0.04207568364801142
wandb: 	momentum: 0.9
wandb: 	net: SimpleCnn
wandb: 	optimizer: adam
wandb: 	use_weighted_sampler: False
wandb: 	weight_decay: 0.02168547243887133


Checking class label counts in train_loader:
Class 0 count: 344
Class 1 count: 356
--------------------


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
test_accuracy,█▁▁▁▁▁▁▁▁███████████████████████████████
test_f1,█▁▁▁▁▁██████████████████████████████████
test_loss,██▇▆▅▃▁▁▁▁▂▂▄▃▃▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▁▁▁▁▂▂▂▂▂▂
train_accuracy,▅█▂▁▁███████████████████████████████████
train_f1,▂█▇▁▁▁▁█████████████████████████████████
train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99
test_accuracy,0.50333
test_f1,0.66962
test_loss,0.69318


In [ ]:
sweep_config = {
    'name': 'sweep_len200_500_n1000',
    'method': 'random',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
        }
    }

parameters_dict = {
    'net': {
        'value': 'SimpleCnnWithPool'
        },
    'layer_number': {
        'values': [2, 3, 4]
    },
    'use_weighted_sampler': {
        'value': False
    },
    "file_name": {
        "value": "len200_500_n1000"
        },
    "optimizer": {
        "value": "adam",
        },
    "learning_rate": {
        "distribution": "log_uniform_values",
        "min": 0.0005,
        "max": 0.05
        },
    "weight_decay": {
        "distribution": "log_uniform_values",
        "min": 1e-5,
        "max": 1e-1
        },
    "dropout": {
        "distribution": "uniform",
        "min": 0.2,
        "max": 0.5
        },
    "batch_size": {
        "values": [128, 256, 512]
        },
    "epochs": {
      "value": 100
      },
    "momentum": {
        "value": None,
        }
    }

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="deep-learning-MBF4")
wandb.agent(sweep_id, train_net, count=100)

Create sweep with ID: g21gpaba
Sweep URL: https://wandb.ai/n-j-hindriks-wur/deep-learning-MBF4/sweeps/g21gpaba


wandb: Agent Starting Run: kfj41ih0 with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.3318777650339785
wandb: 	epochs: 100
wandb: 	file_name: len200_500_n1000
wandb: 	layer_number: 2
wandb: 	learning_rate: 0.0050277420280863665
wandb: 	momentum: None
wandb: 	net: SimpleCnnWithPool
wandb: 	optimizer: adam
wandb: 	use_weighted_sampler: False
wandb: 	weight_decay: 5.4267340689252384e-05


Checking class label counts in train_loader:
Class 0 count: 365
Class 1 count: 335
--------------------


epoch,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██
test_accuracy,▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▃▃▂▂▂▂▃▃▄▅▅▆▅▆▅▆▆▆▆▇▇▇███
test_f1,▁▂▃▃▃▄▄▅▅▅▄▅▅▄▅▆▆▅▆▆▆▇▆▆▇▇▆▆▇▇▇▇▇▇▇▇█▇██
test_loss,▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█████▇▇▇▇▆▇▇▆▆▅▄▄▃▂▃▃▂▁
train_accuracy,▁▁▁▂▁▂▂▂▂▃▃▃▄▃▃▄▅▄▅▅▅▆▆▅▅▅▅▆▅▆▆▅▆▆▆▇██▇█
train_f1,▁▃▃▄▃▄▃▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇█▇████
train_loss,███████▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▅▆▅▅▅▄▄▅▄▄▃▃▃▃▃▂▂▁
epoch,99
test_accuracy,0.66
test_f1,0.62774
test_loss,0.624


wandb: Agent Starting Run: po9g9x76 with config:
wandb: 	batch_size: 512
wandb: 	dropout: 0.22160760115240383
wandb: 	epochs: 100
wandb: 	file_name: len200_500_n1000
wandb: 	layer_number: 4
wandb: 	learning_rate: 0.003839811652730808
wandb: 	momentum: None
wandb: 	net: SimpleCnnWithPool
wandb: 	optimizer: adam
wandb: 	use_weighted_sampler: False
wandb: 	weight_decay: 0.02078887911914654


...

Checking class label counts in train_loader:
Class 0 count: 351
Class 1 count: 349
--------------------


epoch,▁▁▁▁▁▂▂▂▂▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
test_accuracy,▁▁▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂▂▃▃▃▄▅▆▆▇▇▇▇▇████████
test_f1,▆▁▁▁▁▅▅▅▅▄▄▄▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇████████████
test_loss,███████████████████████▇▇▇▇▆▆▄▄▄▃▃▃▂▂▂▁▁
train_accuracy,▁▁▁▁▁▁▂▁▂▁▂▂▁▁▁▂▂▂▃▃▃▃▃▄▃▅▅▆▇▇▇▇█▇██████
train_f1,▄▂▁▃▃▃▃▄▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇███████
train_loss,███████████████████████▇▇▇▇▇▇▆▅▄▃▃▃▃▂▂▁▁
epoch,99
test_accuracy,0.97667
test_f1,0.97561
test_loss,0.2095


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: cajoo5m5 with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.25056866464758054
wandb: 	epochs: 100
wandb: 	file_name: len200_500_n1000
wandb: 	layer_number: 4
wandb: 	learning_rate: 0.0013060560628140486
wandb: 	momentum: None
wandb: 	net: SimpleCnnWithPool
wandb: 	optimizer: adam
wandb: 	use_weighted_sampler: False
wandb: 	weight_decay: 0.0011903004982158655


Checking class label counts in train_loader:
Class 0 count: 363
Class 1 count: 337
--------------------


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇███
test_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
test_f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██
test_loss,█▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▄▃▃▂▁
train_accuracy,▂▂▁▂▂▂▁▁▂▂▁▂▂▂▁▂▂▁▁▁▂▁▁▁▂▂▁▂▁▂▂▃▂▃▃▄▄▅▅█
train_f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▂▁▂▁▂▃▃▄▃▄▅▅▅▇█
train_loss,█▇▇▆▆▇▇▆▇▇▇▇▆▆▇▆▇▆▅▆▆▇▅▇█▅▅▇▆▄▅▅▆▃▄▃▄▃▅▁
epoch,99
test_accuracy,0.49
test_f1,0.02548
test_loss,0.6912


In [ ]:
sweep_config = {
    'name': 'sweep_len200_500_n1000_nr2',
    'method': 'random',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
        }
    }

parameters_dict = {
    'net': {
        'value': 'SimpleCnnWithPool'
        },
    'layer_number': {
        'values': [2, 3, 4]
    },
    'use_weighted_sampler': {
        'value': False
    },
    "file_name": {
        "value": "len200_500_n1000_nr2"
        },
    "optimizer": {
        "value": "adam",
        },
    "learning_rate": {
        'values': [0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05]
        },
    "weight_decay": {
        'values': [0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05]
        },
    "dropout": {
        'values': [0.2, 0.3, 0.4, 0.5]
        },
    "batch_size": {
        "values": [128, 256, 512]
        },
    "epochs": {
      "value": 100
      },
    "momentum": {
        "value": None,
        }
    }

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="deep-learning-MBF4")
wandb.agent(sweep_id, train_net, count=100)

KeyboardInterrupt: 

Results of sweep were a bit dissapointing, weight decay seemed to be too high.

In [ ]:
sweep_config = {
    'name': 'sweep_len200_500_n1000_nr3',
    'method': 'random',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
        }
    }

parameters_dict = {
    'net': {
        'value': 'SimpleCnnWithPool'
        },
    'layer_number': {
        'values': [2, 3, 4]
    },
    'use_weighted_sampler': {
        'value': False
    },
    "file_name": {
        "value": "len200_500_n1000_nr3"
        },
    "optimizer": {
        "value": "adam",
        },
    "learning_rate": {
        'values': [0.005, 0.01, 0.02]
        },
    "weight_decay": {
        'values': [0.1, 0.01, 0.001, 0.0001]
        },
    "dropout": {
        'value': 0.3
        },
    "batch_size": {
        "values": [128, 256, 512]
        },
    "epochs": {
      "value": 100
      },
    "momentum": {
        "value": None,
        }
    }

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="deep-learning-MBF4")
wandb.agent(sweep_id, train_net, count=40)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: 0ysvny00
Sweep URL: https://wandb.ai/n-j-hindriks-wur/deep-learning-MBF4/sweeps/0ysvny00


wandb: Agent Starting Run: 5en30hso with config:
wandb: 	batch_size: 512
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	file_name: len200_500_n1000_nr3
wandb: 	layer_number: 3
wandb: 	learning_rate: 0.02
wandb: 	momentum: None
wandb: 	net: SimpleCnnWithPool
wandb: 	optimizer: adam
wandb: 	use_weighted_sampler: False
wandb: 	weight_decay: 0.1
wandb: Currently logged in as: n-j-hindriks (n-j-hindriks-wur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Checking class label counts in train_loader:
Class 0 count: 1762
Class 1 count: 1738
--------------------


...

Checking class label counts in train_loader:
Class 0 count: 1791
Class 1 count: 1709
--------------------


epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇█
test_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,▁▃▄▇▇████▆▅▅▅▅▆▆▆▆▆▆▅▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▆▃▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99
test_accuracy,0.47867
test_f1,0
test_loss,0.69442


wandb: Agent Starting Run: bsomin0q with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	file_name: len200_500_n1000_nr3
wandb: 	layer_number: 2
wandb: 	learning_rate: 0.02
wandb: 	momentum: None
wandb: 	net: SimpleCnnWithPool
wandb: 	optimizer: adam
wandb: 	use_weighted_sampler: False
wandb: 	weight_decay: 0.0001


Checking class label counts in train_loader:
Class 0 count: 1774
Class 1 count: 1726
--------------------


epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
test_accuracy,▄▁▁▁▁▃▁▂▄▅▄▅▄▆▅▆▆▅▆▅▄▄▅▅▄▃▄▄▅▅▆▆▇▇▆▅▅▅▅█
test_f1,█▁▁▁▁▁▁▁▃▄▅▅▆▆▆▇▆▆▆▇▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
test_loss,▁▁▁▁▁▂▁▄▄▃▄▆▄▆▅▆█▅▆▆▆▆▆▇▇▅▆▇▆▅▆▆▆▇▇▅▅▆▆█
train_accuracy,▁▁▁▃▃▅▄▅▅▅▅▆▅▅▆▇▆▆▆▇▆▆█▇▇▇▇▇▇▆▇▇▇▇██▇▇▇▇
train_f1,▄▁▁▁▄▅▆▆▇▇▇▇▇█▇█▇▇████▇▇████████▇███████
train_loss,█████▇▇▇▇▇▆▅▅▅▅▅▄▄▄▄▃▃▂▃▃▃▂▃▂▃▂▂▂▂▂▁▁▁▂▂
epoch,99
test_accuracy,0.52533
test_f1,0.5908
test_loss,0.70078


A bunch of runs that are doing well (reach high f score before 40 epochs) are runs with 4 layers, a learning rate of 0.01/0.02

In [ ]:
sweep_config = {
    'name': 'sweep_human_training_GO_3A0005576',
    'method': 'grid',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
        }
    }

parameters_dict = {
    'net': {
        'value': 'SimpleCnnWithPool'
        },
    'layer_number': {
        'value': 4
    },
    'use_weighted_sampler': {
        'values':  [True, False]
    },
    "file_name": {
        "value": "human_training_GO_3A0005576"
        },
    "optimizer": {
        "value": "adam",
        },
    "learning_rate": {
        'values': [0.01, 0.02]
        },
    "weight_decay": {
        'value': 0.0001
        },
    "dropout": {
        'value': 0.3
        },
    "batch_size": {
        "value": 512
        },
    "epochs": {
      "value": 100
      },
    "momentum": {
        "value": None,
        }
    }

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="deep-learning-MBF4")
wandb.agent(sweep_id, train_net)

Create sweep with ID: sp6g8n1d
Sweep URL: https://wandb.ai/n-j-hindriks-wur/deep-learning-MBF4/sweeps/sp6g8n1d


wandb: Agent Starting Run: alx99xl3 with config:
wandb: 	batch_size: 512
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	file_name: human_training_GO_3A0005576
wandb: 	layer_number: 4
wandb: 	learning_rate: 0.01
wandb: 	momentum: None
wandb: 	net: SimpleCnnWithPool
wandb: 	optimizer: adam
wandb: 	use_weighted_sampler: True
wandb: 	weight_decay: 0.0001


216
4749
Checking class label counts in train_loader:
Class 0 count: 2304
Class 1 count: 2445
--------------------


epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
test_accuracy,██▇▄▃▁▂▂▂▃▃▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▆▆▇▇▇▇▇▇▇▇▇▇
test_f1,▁▃▃▃▃▃▄▄▄▄▅▆▅▅▆▆▆▆▆▆▆▆▆▇▆▇▇▇▇█▇███████▇▇
test_loss,▆▇▇▇███▆▇▅▅▅▆▆▅▅▆▆▅▅▆▅▅▄▄▄▅▄▃▄▂▂▂▁▁▁▁▁▁▂
train_accuracy,▁▂▃▃▃▄▄▄▆▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
train_f1,▆▁▂▄▄▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████████████
train_loss,█████▇▇▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▂▁▁
epoch,99
test_accuracy,0.85553
test_f1,0.29327
test_loss,0.42926


wandb: Agent Starting Run: oxg350n7 with config:
wandb: 	batch_size: 512
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	file_name: human_training_GO_3A0005576
wandb: 	layer_number: 4
wandb: 	learning_rate: 0.01
wandb: 	momentum: None
wandb: 	net: SimpleCnnWithPool
wandb: 	optimizer: adam
wandb: 	use_weighted_sampler: False
wandb: 	weight_decay: 0.0001


...

epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇██
test_accuracy,▁▁██████████▁▁▁▁▁▁▁▁███████▁▁▁▁▁▁▁▁▁████
test_f1,█▁▁▁▁▁▁▁▁▁████████▁▁▁▁▁▁▁▁██████████▁▁▁█
test_loss,█▄▂▁▁▁▁▁▁▁▂▂▂▂▂▃▂▂▂▂▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
train_accuracy,▂▄▄▇▅▃▄▅▃▃▂█▇▆▅▃▃▂▄▂▅▄▇▆▁▅▁▆▅▃▃▄▂██▇▆▃▅▄
train_f1,▂▁▁▁▁▁▁▂████████▁▁▁▁▁▁▁▁███████████▁▁▁▁█
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99
test_accuracy,0.04472
test_f1,0.08561
test_loss,0.69417


wandb: Agent Starting Run: toocc07y with config:
wandb: 	batch_size: 512
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	file_name: human_training_GO_3A0005576
wandb: 	layer_number: 4
wandb: 	learning_rate: 0.02
wandb: 	momentum: None
wandb: 	net: SimpleCnnWithPool
wandb: 	optimizer: adam
wandb: 	use_weighted_sampler: False
wandb: 	weight_decay: 0.0001


Checking class label counts in train_loader:
Class 0 count: 4532
Class 1 count: 217
--------------------


epoch,▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇██
test_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,█▆▁▁▁▂▃▂▁▁▂▂▂▂▁▁▁▁▁▂▂▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_f1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▃▂▄▄▁▂▂▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99
test_accuracy,0.95577
test_f1,0
test_loss,0.18698


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [ ]:
sweep_config = {
    'name': 'human_training_over_GOs',
    'method': 'grid',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
        }
    }

parameters_dict = {
    'net': {
        'value': 'SimpleCnnWithPool'
        },
    'layer_number': {
        'value': 4
    },
    'use_weighted_sampler': {
        'value':  True
    },
    "file_name": {
        "values": ["human_training_GO_3A0005576",
                   "human_training_GO_3A0005739",
                   "human_training_GO_3A0007165",
                   "human_training_GO_3A0043066",
                   "human_training_GO_3A0055085"]
        },
    "optimizer": {
        "value": "adam",
        },
    "learning_rate": {
        'values': [0.01, 0.02]
        },
    "weight_decay": {
        'value': 0.0001
        },
    "dropout": {
        'value': 0.3
        },
    "batch_size": {
        "value": 512
        },
    "epochs": {
      "value": 100
      },
    "momentum": {
        "value": None,
        }
    }

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="deep-learning-MBF4")
wandb.agent(sweep_id, train_net)

Create sweep with ID: xzkvi0wy
Sweep URL: https://wandb.ai/n-j-hindriks-wur/deep-learning-MBF4/sweeps/xzkvi0wy


wandb: Agent Starting Run: bq2j23my with config:
wandb: 	batch_size: 512
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	file_name: human_training_GO_3A0005576
wandb: 	layer_number: 4
wandb: 	learning_rate: 0.01
wandb: 	momentum: None
wandb: 	net: SimpleCnnWithPool
wandb: 	optimizer: adam
wandb: 	use_weighted_sampler: True
wandb: 	weight_decay: 0.0001


231
4749
Checking class label counts in train_loader:
Class 0 count: 2389
Class 1 count: 2360
--------------------


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇████
test_accuracy,▁▁▁▁▁███▇▆▆▅▅▅▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
test_f1,▁▁▁▁▁▃▃▂▁▂▄▅▇▇▇▆▇▇▇▇▇▇▇▆▆▇▆▆▇▇▇██▇▇█▇███
test_loss,████████▇▇▇▇▇▆▅▃▅▄▄▄▃▃▄▄▄▃▃▂▂▃▂▃▂▂▃▁▂▂▂▁
train_accuracy,▁▁▁▁▁▁▁▂▃▃▃▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█████████
train_f1,▂▆▁▁▂▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████████████████
train_loss,███████████▇▇▇▆▅▅▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
epoch,99
test_accuracy,0.86683
test_f1,0.28496
test_loss,0.29359


wandb: Agent Starting Run: i1d9omvy with config:
wandb: 	batch_size: 512
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	file_name: human_training_GO_3A0005576
wandb: 	layer_number: 4
wandb: 	learning_rate: 0.02
wandb: 	momentum: None
wandb: 	net: SimpleCnnWithPool
wandb: 	optimizer: adam
wandb: 	use_weighted_sampler: True
wandb: 	weight_decay: 0.0001


...

epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
test_accuracy,█▃▁▁▃▂▂▃▄▄▆▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇█▇████▇▇█▇█▇█
test_f1,▂▁▁▃▂▃▂▃▃▃▃▃▃▄▆▆▆▆▆▇▆▆▇▇▇▇▇▇▇█▇▇▇▇▇▆▇█▆█
test_loss,▇▇██▆▇▆▇▆▆▅▄▃▅▂▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▁▂▁▂▁▂▁▂▂▁
train_accuracy,▁▃▃▄▄▄▄▅▅▆▆▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇▇█▇███
train_f1,▆▂▁▄▆▆▆▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
train_loss,██████▇▇▇▇▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▁▁▁▁▁▁
epoch,99
test_accuracy,0.94349
test_f1,0.39791
test_loss,0.25505


wandb: Agent Starting Run: 2efx2180 with config:
wandb: 	batch_size: 512
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	file_name: human_training_GO_3A0055085
wandb: 	layer_number: 4
wandb: 	learning_rate: 0.02
wandb: 	momentum: None
wandb: 	net: SimpleCnnWithPool
wandb: 	optimizer: adam
wandb: 	use_weighted_sampler: True
wandb: 	weight_decay: 0.0001


157
4749
Checking class label counts in train_loader:
Class 0 count: 2371
Class 1 count: 2378
--------------------


epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇██
test_accuracy,▁██▂▄▆▆▆█▇██▇████▇▇█████████████████████
test_f1,▁▁▁▂▃▄▆▆▆▆▇▆▅▇▇▆█▇▇▇██▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇█▇
test_loss,▇▇▇▇█▇▇▇▇▆▆▆▆▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▂
train_accuracy,▁▁▁▃▃▃▄▄▄▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█▇██████████
train_f1,▁▄▄▃▃▅▅▅▆▆▆▆▇▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇███▇▇███████
train_loss,██▇▇▇▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▂▁
epoch,99
test_accuracy,0.92776
test_f1,0.36364
test_loss,0.30858


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [ ]:
def train_net_focal_loss(config=None):
    """Train the network with specified parameters."""


    # Initialize wandb
    with wandb.init(config=config) as run:
      config = wandb.config

      seq_file = file_names[config.file_name]['seq']
      pos_file = file_names[config.file_name]['pos']


      lr = round(config.learning_rate, 3)
      # # Create a custom name from hyperparameters
      run.name = f"lr_{lr}_ln_{config.layer_number}"
      run.save()  # This makes sure the name is visible on the dashboard

      # Set device
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

      # Load data
      dataset = read(seq_file, pos_file)
      train_data, test_data = generate_train_test(*dataset)

      max_len = 0
      for seq in train_data[0]:
        if len(seq) > max_len:
          max_len = len(seq)

      if config.use_weighted_sampler:
        print(sum(train_data[1]))
        class_sample_count = np.array([len(np.where(np.array(train_data[1]) == t)[0]) for t in np.unique(train_data[1])])
        weight = 1. / class_sample_count
        samples_weight = np.array([weight[t] for t in train_data[1]])
        samples_weight = torch.from_numpy(samples_weight)
        print(len(samples_weight))
        sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight), replacement=True)

        train_loader = load_data(config.batch_size, max_len, train_data, sampler=sampler)
      else:
        train_loader = load_data(config.batch_size, max_len, train_data)
      test_loader  = load_data(config.batch_size, max_len, test_data)

      print("Checking class label counts in train_loader:")
      label_counts = {0: 0, 1: 0}  # Initialize counts for each class
      for _, labels in train_loader:
          for label in labels.flatten():
              label_counts[label.item()] += 1
      print(f"Class 0 count: {label_counts[0]}")
      print(f"Class 1 count: {label_counts[1]}")
      print("-" * 20)  # Separator

      class_weights = torch.tensor([1 / label_counts[0], 1 / label_counts[1]], device=device)
      model = architectures[config.net](config.dropout, number_of_layers=config.layer_number, focal_alpha=class_weights, gamma=config.gamma).to(device)
      optimizer = get_optimizer(model, config.optimizer, lr, config.momentum)

      for epoch in range(config.epochs):
          # Training loop for one epoch
          model.train()  # Set the model to training mode
          all_x = []
          all_y = []
          for x, y in train_loader:
              all_x.append(x.to(device))
              all_y.append(y.to(device))

          all_x = torch.cat(all_x, dim=0)
          all_y = torch.cat(all_y, dim=0)

          optimizer.zero_grad()  # Reset gradients
          pred, loss, accuracy = model(all_x, all_y)  # Forward pass for all data
          loss.backward()  # Backpropagate gradients
          optimizer.step()  # Update model parameters

          predicted_labels = (pred > 0.5).float()
          train_f1 = f1_score(all_y.cpu().numpy(), predicted_labels.cpu().numpy())

          all_y_np = all_y.cpu().numpy()
          predicted_labels_np = predicted_labels.cpu().numpy()

          cm = confusion_matrix(all_y_np, predicted_labels_np)
          TN, FP, FN, TP = cm.ravel()

          # Evaluation after each epoch
          model.eval()  # Set the model to evaluation mode
          with torch.no_grad():  # Disable gradient calculation
              all_test_x = []
              all_test_y = []
              for x, y in test_loader:
                  all_test_x.append(x.to(device))
                  all_test_y.append(y.to(device))

              all_test_x = torch.cat(all_test_x, dim=0)
              all_test_y = torch.cat(all_test_y, dim=0)

              test_pred, test_loss, test_accuracy = model(all_test_x, all_test_y)

          predicted_labels = (test_pred > 0.5).float()
          test_f1 = f1_score(all_test_y.cpu().numpy(), predicted_labels.cpu().numpy())

          all_test_y_np = all_test_y.cpu().numpy()
          predicted_labels_np = predicted_labels.cpu().numpy()

          cm_test = confusion_matrix(all_test_y_np, predicted_labels_np)
          TN_test, FP_test, FN_test, TP_test = cm_test.ravel()

          wandb.log({
              "epoch" : epoch,
              "train_loss": loss.item(),
              "test_loss": test_loss.item(),
              "train_accuracy": accuracy.item(),
              "test_accuracy": test_accuracy.item(),
              "train_f1": train_f1,
              "test_f1": test_f1,
              "fp_rate_train": FP / (FP + TN),
              "fp_rate_test": FP_test / (FP_test + TN_test),
              "fn_rate_train": FN / (TP + FN),
              "fn_rate_test": FN_test / (TP_test + FN_test)
          })


In [ ]:
architectures = {
    'SimpleCnn': SimpleCnn,
    'SimpleCnnWithPool': SimpleCnnWithPool,
    'SimpleCnnWithPool_FocalLoss': SimpleCnnWithPool_FocalLoss
}

sweep_config = {
    'name': 'sweep_human_data_FocalLoss_gamma',
    'method': 'grid',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
        }
    }

parameters_dict = {
    'net': {
        'value': 'SimpleCnnWithPool_FocalLoss'
        },
    'layer_number': {
        'value': 4
    },
    'use_weighted_sampler': {
        'value': False
    },
    "file_name": {
        "value": "human_training_GO_3A0005576"
        },
    "optimizer": {
        "value": "adam",
        },
    "learning_rate": {
        'value': 0.01
        },
    "weight_decay": {
        'value': 0.0001
        },
    "dropout": {
        'value': 0.3
        },
    "batch_size": {
        "value": 512
        },
    "epochs": {
      "value": 100
      },
    "momentum": {
        "value": None,
        },
    "gamma": {
        "values": [0, 1, 2, 3, 4, 5]
        }
    }

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="deep-learning-MBF4")
wandb.agent(sweep_id, train_net_focal_loss)

KeyboardInterrupt: 

## Try decreasing learning rate stepwise




In [ ]:
def train_net_focal_loss_decreasing_lr(config=None):
    """Train the network with specified parameters."""


    # Initialize wandb
    with wandb.init(config=config) as run:
      config = wandb.config

      seq_file = file_names[config.file_name]['seq']
      pos_file = file_names[config.file_name]['pos']


      lr = round(config.learning_rate, 3)
      # # Create a custom name from hyperparameters
      run.name = f"lr_{lr}_ln_{config.layer_number}"
      run.save()  # This makes sure the name is visible on the dashboard

      # Set device
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

      # Load data
      dataset = read(seq_file, pos_file)
      train_data, test_data = generate_train_test(*dataset)

      max_len = 0
      for seq in train_data[0]:
        if len(seq) > max_len:
          max_len = len(seq)

      if config.use_weighted_sampler:
        print(sum(train_data[1]))
        class_sample_count = np.array([len(np.where(np.array(train_data[1]) == t)[0]) for t in np.unique(train_data[1])])
        weight = 1. / class_sample_count
        samples_weight = np.array([weight[t] for t in train_data[1]])
        samples_weight = torch.from_numpy(samples_weight)
        print(len(samples_weight))
        sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight), replacement=True)

        train_loader = load_data(config.batch_size, max_len, train_data, sampler=sampler)
      else:
        train_loader = load_data(config.batch_size, max_len, train_data)
      test_loader  = load_data(config.batch_size, max_len, test_data)

      print("Checking class label counts in train_loader:")
      label_counts = {0: 0, 1: 0}  # Initialize counts for each class
      for _, labels in train_loader:
          for label in labels.flatten():
              label_counts[label.item()] += 1
      print(f"Class 0 count: {label_counts[0]}")
      print(f"Class 1 count: {label_counts[1]}")
      print("-" * 20)  # Separator

      class_weights = torch.tensor([1 / label_counts[0], 1 / label_counts[1]], device=device)
      model = architectures[config.net](config.dropout, number_of_layers=config.layer_number, focal_alpha=class_weights, gamma=config.gamma, final_output_number=config.final_output_number).to(device)
      optimizer = get_optimizer(model, config.optimizer, lr, config.momentum)
      scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config.scheluder_step, gamma=config.scheluder_gamma)

      for epoch in range(config.epochs):
          # Training loop for one epoch
          model.train()  # Set the model to training mode
          all_x = []
          all_y = []
          for x, y in train_loader:
              all_x.append(x.to(device))
              all_y.append(y.to(device))

          all_x = torch.cat(all_x, dim=0)
          all_y = torch.cat(all_y, dim=0)

          optimizer.zero_grad()  # Reset gradients
          pred, loss, accuracy = model(all_x, all_y)  # Forward pass for all data
          loss.backward()  # Backpropagate gradients
          optimizer.step()  # Update model parameters
          scheduler.step()

          predicted_labels = (pred > 0.5).float()
          train_f1 = f1_score(all_y.cpu().numpy(), predicted_labels.cpu().numpy())

          cm = confusion_matrix(all_y, predicted_labels)
          TN, FP, FN, TP = cm.ravel()

          # Evaluation after each epoch
          model.eval()  # Set the model to evaluation mode
          with torch.no_grad():  # Disable gradient calculation
              all_test_x = []
              all_test_y = []
              for x, y in test_loader:
                  all_test_x.append(x.to(device))
                  all_test_y.append(y.to(device))

              all_test_x = torch.cat(all_test_x, dim=0)
              all_test_y = torch.cat(all_test_y, dim=0)

              test_pred, test_loss, test_accuracy = model(all_test_x, all_test_y)

          predicted_labels = (test_pred > 0.5).float()
          test_f1 = f1_score(all_test_y.cpu().numpy(), predicted_labels.cpu().numpy())

          cm_test = confusion_matrix(all_test_y, predicted_labels)
          TN_test, FP_test, FN_test, TP_test = cm_test.ravel()

          wandb.log({
              "epoch" : epoch,
              "train_loss": loss.item(),
              "test_loss": test_loss.item(),
              "train_accuracy": accuracy.item(),
              "test_accuracy": test_accuracy.item(),
              "train_f1": train_f1,
              "test_f1": test_f1,
              "fp_rate_train": FP / (FP + TN),
              "fp_rate_test": FP_test / (FP_test + TN_test),
              "fn_rate_train": FN / (TP + FN),
              "fn_rate_test": FN_test / (TP_test + FN_test)
          })


In [ ]:
architectures = {
    'SimpleCnn': SimpleCnn,
    'SimpleCnnWithPool': SimpleCnnWithPool,
    'SimpleCnnWithPool_FocalLoss': SimpleCnnWithPool_FocalLoss
}

sweep_config = {
    'name': 'sweep_human_data_decreasing_lr',
    'method': 'grid',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
        }
    }

parameters_dict = {
    'net': {
        'value': 'SimpleCnnWithPool_FocalLoss'
        },
    'layer_number': {
        'value': 4
    },
    'use_weighted_sampler': {
        'value': False
    },
    "file_name": {
        "value": "human_training_GO_3A0005576"
        },
    "optimizer": {
        "value": "adam",
        },
    "learning_rate": {
        'value': 0.01
        },
    "weight_decay": {
        'value': 0.0001
        },
    "dropout": {
        'value': 0.3
        },
    "batch_size": {
        "value": 512
        },
    "epochs": {
      "value": 100
      },
    "momentum": {
        "value": None,
        },
    "gamma": {
        "values": [1, 3, 5]
        },
    "scheluder_step": {
        "value": 20
        },
    "scheluder_gamma": {
        "value": 0.5
        },
    "final_output_number": {
        "values": [3, 4, 5]
    }
    }


sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="deep-learning-MBF4")
wandb.agent(sweep_id, train_net_focal_loss_decreasing_lr)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: ofyzez9v
Sweep URL: https://wandb.ai/n-j-hindriks-wur/deep-learning-MBF4/sweeps/ofyzez9v


wandb: Agent Starting Run: yo2syovg with config:
wandb: 	batch_size: 512
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	file_name: human_training_GO_3A0005576
wandb: 	final_output_number: 3
wandb: 	gamma: 1
wandb: 	layer_number: 4
wandb: 	learning_rate: 0.01
wandb: 	momentum: None
wandb: 	net: SimpleCnnWithPool_FocalLoss
wandb: 	optimizer: adam
wandb: 	scheluder_gamma: 0.5
wandb: 	scheluder_step: 20
wandb: 	use_weighted_sampler: False
wandb: 	weight_decay: 0.0001
wandb: Currently logged in as: n-j-hindriks (n-j-hindriks-wur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Checking class label counts in train_loader:
Class 0 count: 4529
Class 1 count: 220
--------------------


...

epoch,▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇█████
fn_rate_test,█████▆▄▄▄▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
fn_rate_train,▃▄███▇▄▃▄▅▄▃▃▃▃▂▁▁▁▁▁▁▂▁▁▁▂▁▂▂▁▁▁▁▁▂▁▁▂▁
fp_rate_test,▁▁▁▁▃▄▄▄███▆▇▆▅▅▅▄▅▅▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
fp_rate_train,█▃▁▁▂▄▄▃▃▄▄▄▃▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
test_accuracy,▁█████▇▅▅▅▁▁▃▄▅▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
test_f1,▁▄▁▁▅▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇████▇█
test_loss,█████▇████▇▇▆▆▅▃▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▅██▇▃▂▃▂▂▂▂▂▁▁▂▂▃▃▄▄▅▄▄▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆
train_f1,▃▂▂▁▃▃▃▃▄▄▄▄▄▄▄▅▆▆▆▆▆▆▇▇▇██▇▇▇█▇▇▇█▇▇█▇█
train_loss,██████▇▇▇▇▆▅▄▅▄▃▃▃▂▃▂▂▂▁▂▂▂▂▂▂▂▁▂▁▁▂▂▁▁▁


wandb: Agent Starting Run: 4qyq4lgh with config:
wandb: 	batch_size: 512
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	file_name: human_training_GO_3A0005576
wandb: 	final_output_number: 5
wandb: 	gamma: 5
wandb: 	layer_number: 4
wandb: 	learning_rate: 0.01
wandb: 	momentum: None
wandb: 	net: SimpleCnnWithPool_FocalLoss
wandb: 	optimizer: adam
wandb: 	scheluder_gamma: 0.5
wandb: 	scheluder_step: 20
wandb: 	use_weighted_sampler: False
wandb: 	weight_decay: 0.0001


Checking class label counts in train_loader:
Class 0 count: 4528
Class 1 count: 221
--------------------


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇██
fn_rate_test,████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
fn_rate_train,▁▁▆██▃▅▅▆▆▄▄▄▄▄▃▄▄▃▄▄▄▄▅▄▄▄▄▄▄▄▄▅▅▅▅▄▅▄▅
fp_rate_test,██▁▁▁▁▁▁▁▁▁█████████████████████████████
fp_rate_train,▂▁▁▁█▆▄▄▃▃▆▆▆▆▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▅▅▆▅▆▆
test_accuracy,█████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_f1,█▁▁▁▁▁▁▁████████████████████████████████
test_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▇▁▆█▃▆▆▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▅▄▅▅▅▅▅▅▅▅▄▄
train_f1,▄▇▁▁▇▇▇▇▆▇▇███▇▇▇▇▇▇█▇▇▇▇▇▇█▇█▇▇▇██▇█▇▇█
train_loss,█▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## Best SimpleCnn on human data

In [ ]:
architectures = {
    'SimpleCnn': SimpleCnn,
    'SimpleCnnWithPool': SimpleCnnWithPool,
    'SimpleCnnWithPool_FocalLoss': SimpleCnnWithPool_FocalLoss
}

sweep_config = {
    'name': 'sweep_human_data_all_SimpleCnn',
    'method': 'grid',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
        }
    }

# Best parameters for simple CNN, but setting use_weighted_sampler to true, because of class imbalance
# Best run on sample data: https://wandb.ai/n-j-hindriks-wur/deep-learning-MBF4/runs/rrjj8lep/overview
parameters_dict = {
    'net': {
        'value': 'SimpleCnn'
        },
    'layer_number': {
        'value': 5
    },
    'use_weighted_sampler': {
        'value': True
    },
    "file_name": {
        "values": ["human_training_GO_3A0005576",
                   "human_training_GO_3A0005739",
                   "human_training_GO_3A0007165",
                   "human_training_GO_3A0043066",
                   "human_training_GO_3A0055085"]
        },
    "optimizer": {
        "value": "adam",
        },
    "learning_rate": {
        'value': 0.06
        },
    "weight_decay": {
        'value': 0.067
        },
    "dropout": {
        'value': 0.4
        },
    "batch_size": {
        "value": 512
        },
    "epochs": {
      "value": 100
      },
    "momentum": {
        "value": None,
        }
    }


sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="deep-learning-MBF4")
wandb.agent(sweep_id, train_net)


Create sweep with ID: 3xivsc3s
Sweep URL: https://wandb.ai/dom20020706-wageningen-university-and-research/deep-learning-MBF4/sweeps/3xivsc3s


wandb: Agent Starting Run: 50mcgd8p with config:
wandb: 	batch_size: 512
wandb: 	dropout: 0.4
wandb: 	epochs: 100
wandb: 	file_name: human_training_GO_3A0005576
wandb: 	layer_number: 5
wandb: 	learning_rate: 0.06
wandb: 	momentum: None
wandb: 	net: SimpleCnn
wandb: 	optimizer: adam
wandb: 	use_weighted_sampler: True
wandb: 	weight_decay: 0.067
wandb: Currently logged in as: dom20020706 (dom20020706-wageningen-university-and-research) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


227
4749
Checking class label counts in train_loader:
Class 0 count: 2380
Class 1 count: 2369
--------------------


epoch,▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇█
test_accuracy,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁█████████████▁▁▁▁████▁▁██
test_f1,████████████▁▁▁▁▁▁▁▁▁▁▁▁▁██████▁▁▁▁▁▁██▁
test_loss,▁███████████████████████████████████████
train_accuracy,▄▄▆▇▄▁▄▆▆▃▆▄▅▄▄▄▄█▃▇▆▆▄▄▆▅▅▅▃▆▅▄▅▅▄▄▃▅▄▆
train_f1,▁▁██████████▁▁▁▁▁▁▁▁▁▁▁▁▁████▁▁▁▁▁▁████▁
train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99
test_accuracy,0.96069
test_f1,0
test_loss,0.69092


...

epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇████
test_accuracy,▁██████████████▁▁▁▁▁▁██████████▁▁▁▁▁▁▁▁▁
test_f1,▁▁▁▁▁▁▁▁▁▁██████▁▁▁▁▁▁▁▁▁▁▁▁███████████▁
test_loss,▄▂▂▁▁▁▂▂▂▄▅▅▆▆▇█████▇▇▇▇▇▇▆▆▆▇▇▇▇██████▇
train_accuracy,▃▄▄▅█▄▆▅▃▄▅▅▅▅▆▃▁▄▅▅▆▆▅▅▃▆▃▃▄▅▄▅▃▅▅▃▃▄▄▅
train_f1,▃▁▁▁▁▁▁▁▁▁▁█████████▁▁▁▁▁▁▁▁▁▁█████████▁
train_loss,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99
test_accuracy,0.97494
test_f1,0
test_loss,0.69085


wandb: Agent Starting Run: k8s4n5m8 with config:
wandb: 	batch_size: 512
wandb: 	dropout: 0.4
wandb: 	epochs: 100
wandb: 	file_name: human_training_GO_3A0055085
wandb: 	layer_number: 5
wandb: 	learning_rate: 0.06
wandb: 	momentum: None
wandb: 	net: SimpleCnn
wandb: 	optimizer: adam
wandb: 	use_weighted_sampler: True
wandb: 	weight_decay: 0.067


154
4749
Checking class label counts in train_loader:
Class 0 count: 2313
Class 1 count: 2436
--------------------


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇███
test_accuracy,▁▁███████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁███▁▁▁▁▁▁▁▁▁▁
test_f1,█▁▁▁▁▁▁▁▁▁▁▁▁▁███████████████████▁▁█████
test_loss,▆▁▁▁▁▂▃▃▃▃▅▅▅▆▆▇████▇▇▇▆▆▅▅▄▄▅▆▆▅▄▄▅▆▇▇▆
train_accuracy,▄▆▅▃▆▄▃▃▃▅▂▄▃▄▅▃█▄▁▃▄▅▅▃▄▅▂▃▃▃▄▄▃▆▅▄▅▄▂▃
train_f1,▂▁▁▁▁▁▁▁▁▁▁▁█████████████▁████████▁█████
train_loss,▁█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99
test_accuracy,0.03538
test_f1,0.06834
test_loss,0.69606


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## Best SimpleCnnWithPool_FocalLoss

In [ ]:
architectures = {
    'SimpleCnn': SimpleCnn,
    'SimpleCnnWithPool': SimpleCnnWithPool,
    'SimpleCnnWithPool_FocalLoss': SimpleCnnWithPool_FocalLoss
}

sweep_config = {
    'name': 'sweep_human_data_all_SimpleCnnWithPool_FocalLoss',
    'method': 'grid',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
        }
    }

parameters_dict = {
    'net': {
        'value': 'SimpleCnnWithPool_FocalLoss'
        },
    'layer_number': {
        'value': 4
    },
    'use_weighted_sampler': {
        'value': False
    },
    "file_name": {
        "values": ["human_training_GO_3A0005576",
                   "human_training_GO_3A0005739",
                   "human_training_GO_3A0007165",
                   "human_training_GO_3A0043066",
                   "human_training_GO_3A0055085"]
        },
    "optimizer": {
        "value": "adam",
        },
    "learning_rate": {
        'value': 0.01
        },
    "weight_decay": {
        'value': 0.0001
        },
    "dropout": {
        'value': 0.3
        },
    "batch_size": {
        "value": 512
        },
    "epochs": {
      "value": 100
      },
    "momentum": {
        "value": None,
        },
    "gamma": {
        "value": 3
        }
    }


sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="deep-learning-MBF4")
wandb.agent(sweep_id, train_net_focal_loss)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: p1tvfc8r
Sweep URL: https://wandb.ai/dom20020706-wageningen-university-and-research/deep-learning-MBF4/sweeps/p1tvfc8r


...

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██
fn_rate_test,█████▂▁▂▂███▅▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
fn_rate_train,▁▁▅██▄▃▃▂▃▄▃▄▄▄▃▃▃▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
fp_rate_test,▁▁▁▁▁▇▇▇██▇▄▁▁▁▁▁▅▅▆▇▇▇▇▇▆▆▆▆▆▇▇▇▇▇▇▇▇▆▆
fp_rate_train,▄▁▁▅▆█████▇▆▆▆▆▆▆▆▆▆▆▆▆▅▅▅▆▅▆▆▆▅▅▅▅▅▅▅▅▅
test_accuracy,█████▁███████▃▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
test_f1,▇▇▁▁▁▇▇▇▇▇█▁▁▁▁█▇█▇▇▇▇▇▇▇▆▆▆▇▇▇▇▆▆▇▇▇▇▇▇
test_loss,▃█▁▃▆▇▆▅▅▄▃▃▃▃▃▃▄▄▄▄▃▃▂▂▂▂▂▃▃▃▃▃▂▂▂▃▃▃▃▃
train_accuracy,▁▅███▂▂▂▃▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▅▅▅▄▄▄▄▅
train_f1,▇▁▄▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇▇█▇█▇████▇▇███▇█
train_loss,█▂▃▃▂▂▂▂▁▂▂▂▂▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


wandb: Agent Starting Run: p7lfrvub with config:
wandb: 	batch_size: 512
wandb: 	dropout: 0.3
wandb: 	epochs: 100
wandb: 	file_name: human_training_GO_3A0055085
wandb: 	gamma: 3
wandb: 	layer_number: 4
wandb: 	learning_rate: 0.01
wandb: 	momentum: None
wandb: 	net: SimpleCnnWithPool_FocalLoss
wandb: 	optimizer: adam
wandb: 	use_weighted_sampler: False
wandb: 	weight_decay: 0.0001


Checking class label counts in train_loader:
Class 0 count: 4591
Class 1 count: 158
--------------------


epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
fn_rate_test,▁███▅▄▄▃▄▃▄▄▄▃▄▄▄▄▄▄▄▃▄▃▃▄▄▄▄▄▄▄▄▄▄▄▄▅▄▅
fn_rate_train,▅▁█▆▂██▇▃▃▃▃▄▄▃▃▃▃▃▃▃▃▂▃▃▃▃▃▂▂▃▂▂▃▂▂▂▂▂▁
fp_rate_test,▁▅▆▇▇▃▆█▅▂▂▃▄▂▄▃▃▃▃▂▃▄▂▂▄▃▂▃▂▂▂▂▃▃▂▂▂▂▁▂
fp_rate_train,█▃▂▆▃▂▄▄▃▃▃▃▂▂▂▂▂▂▃▁▃▁▂▁▁▁▂▂▁▂▁▂▂▁▂▂▁▂▂▂
test_accuracy,█▁▁▆▆▅▅▅▆▇▆▇▇▇▇▇▇▇▇▇█▆▇█████▇███████████
test_f1,▁▂▂▂▂▂▃▅▃▄▄▄▄▄▅▅▇▅▆▃▇▆▄▅▅▆▅▆▆▅▆▆▆▆▆▆██▆█
test_loss,████▇▇▇▇▆▆▅▅▄▄▄▃▂▃▂▁▂▂▁▂▁▂▂▁▁▁▁▁▁▂▁▂▂▆▂▅
train_accuracy,▂▁▃▆▇▅▆▄▆▅▆▇▅▆▅▆▆▆▆▆▆▅▇▅▇▇██▇▇▇██▇█▇▇▇█▇
train_f1,▁▁▂▂▂▂▃▃▂▃▄▃▄▄▄▅▅▄▇▅▆█▆█▅▇▅▇▆▇▇▆▇▆█▆▇▆▆▆
train_loss,███████▇▇▇▆▆▆▅▅▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▂▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
